In [1]:
import spb.sdk

# Print Suite SDK version
print(spb.sdk.__version__)

0.14.7


### Connect to Suite Client with Project

In [ ]:
project_name= 'PROJECT_NAME'
team_name = 'TEAM_NAME'
access_key = 'ACCESS_KEY'
client = spb.sdk.Client(project_name=project_name, team_name=team_name, access_key=access_key)
# Print project information
print('Project Name: {}'.format(client.get_project_name()))
print('Total number of data: {}'.format(client.get_num_data()))



### Upload Images

In [ ]:
import glob

# Upload Images first
images_to_upload = glob.glob('./image_path/*.jpg', recursive=True)
# Dataset name is monoclass_dataset, you may set it to something else
for path in images_to_upload:
    client.upload_image(path, 'DATASET_NAME')

### This is for Bounding Boxes

In [ ]:
import json
import glob

# Prep annotations to upload
json_to_upload = glob.glob('./json_path/*.json', recursive=True)
json_dict = {}
for file in json_to_upload:
    with open(file, 'r') as f:
        annotation = json.load(f)
        data_key = annotation['Info']['filename']
        json_dict[data_key] = []
        for obj in annotation['Annotations']:
            class_name = obj['class_code']
            bbox_string = obj['bbox.points']
            bbox = eval(bbox_string)
            # Suite bbox format is x1, y1, width, height
            #    where x1, y1 are the top left coordinates
            x1 = bbox[0]
            y1 = bbox[1]
            width = bbox[2]
            height = bbox[3]
            upload_box = {'coord': {'x': x1, 'y': y1, 'width': width, 'height': height}}
            json_dict[data_key].append({'class_name' : class_name, 'annotation' : upload_box})
            

In [ ]:
# This is a example of a sample json that is used for the code above
{
	"Info": {
		"id": "example_file_name",
		"filename": "example_file_name.jpg",
		"file_folder": None,
		"resolution": "[1536,1024]",
		"region_name": None,
		"date": None,
		"file format": "jpg",
		"copyright": "Y",
		"pixel": "1572864"
	},
	"Annotations": [
		{
			"annotation_id": "3858269",
			"annotation_type": "rect",
			"class_code": "example_bbox",
			"class_name": None,
			"bbox.points": "[1211,424,279,175]",
			"morphic_step": None,
			"spawn": None,
			"memo": None
		}
	]
}

### This is to prep Polygons in COCO format

In [ ]:
import json
import glob

# Prep annotations to upload
json_to_upload = glob.glob('./json_path/*.json', recursive=True)
json_dict = {}
class_id = {}
for file in json_to_upload:
    with open(file, 'r') as f:
        annotation = json.load(f)
        data_key = annotation['images']['file_name']
        json_dict[data_key] = []
        for cat in annotation['categories']:
            class_id[cat['id']] = cat['name']
            
        for obj in annotation['annotations']:
            polygon_to_change = obj['segmentation']
            # Suite polygons are closed multi-polygons
            new_polygons = []
            for ls in polygon_to_change:
                if len(ls) < 1:
                    continue
                new_polygon = [{'x': ls[i], 'y': ls[i+1]} for i in range(0, len(ls)-1, 2)]
                # Since the polygon was a open polygon, we need to add the first x,y points to the end
                new_polygon.append(new_polygon[0])
                new_polygons.append(new_polygon)

            upload_polygon = {'multiple': True, 'coord': {'points' : [new_polygons]}}
            json_dict[data_key].append({'class_name' : class_id[obj['category_id']], 'annotation' : upload_polygon})
            

In [ ]:
# This is a example of a sample json that is used for the code above
{
	"id": "example_image",
	"images": {
		"id": 8,
		"file_name": "example_image.JPG",
		"width": 1300,
		"height": 600
	},
	"type": "instances",
	"annotations": [
		{
			"segmentation": [
				[
					387,
					331,
					403,
					331,
					413,
					334,
					416,
					340,
					415,
					347,
					411,
					350,
					402,
					348,
					395,
					345,
					390,
					343,
					385,
					340,
					385,
					340
				]
			],
			"area": 428,
			"iscrowd": 0,
			"ignore": 0,
			"image_id": 8,
			"category_id": 376,
			"id": 13,
			"bbox": [
				385,
				331,
				31,
				19
			]
		},
		{
			"segmentation": [
				[
					350,
					314,
					373,
					331,
					386,
					342,
					412,
					347,
					445,
					346,
					471,
					339,
					508,
					311,
					528,
					297,
					529,
					286,
					519,
					282,
					501,
					280,
					462,
					277,
					435,
					276,
					415,
					274,
					421,
					285,
					404,
					297,
					383,
					306
				]
			],
			"area": 7928,
			"iscrowd": 0,
			"ignore": 0,
			"image_id": 8,
			"category_id": 375,
			"id": 14,
			"bbox": [
				350,
				274,
				179,
				73
			]
		}
	],
	"categories": [
		{
			"supercategory": "none",
			"id": 375,
			"name": "class_1"
		},
		{
			"supercategory": "none",
			"id": 376,
			"name": "class_2"
		}
	]
}

### You must have set up your Project with all the Classes beforehand

In [ ]:
# now we will go through the labels that we have already uploaded
dataset = 'DATASET_NAME'

# Number of data containing all the tags
num_data = client.get_num_data(dataset=dataset)
num_page = (num_data + 10 - 1) // 10

print('Number of data in the dataset {}: {}'.format(dataset, num_data))

# go through all of the data in the dataset
for page_idx in range(num_page):
    for data_handler in client.get_data_page(page_idx=page_idx, page_size=10, dataset=dataset):
        data_key = data_handler.get_key()
        try:
            annotations = json_dict[data_key]
            for an in annotations:
                data_handler.add_object_label(class_name=an['class_name'], annotation=an['annotation'])
        except:
            print('data_key not found')
                